### 講習5 --- astropy.io.fitsの基本  

FITSデータの読み書きが可能なモジュールです。 元々、pyfitsと呼ばれていたものです。astropyに吸収されてそちらで統一的に管理されるようです。

PyRAFでもFITSデータの読み書きはできるのですが、ndarrayとしてデータを読み出しておくとより自由な処理を行うことができます。例えば、IRAFには存在しない統計アルゴリズムを適用する、あるいは、matplotlibを用いた自由度の高い可視化などです。  

まずastropy.ioからfitsをimportしておきます。

In [1]:
from astropy.io import fits

#### 基本のgetdata
まずは簡単なデータ読み出し方法。 **fits.getdata( )**です。 ピクセル値をndarrayとして読み出します。

In [2]:
data = fits.getdata('idf0011.fits')

In [3]:
data

array([[ 37.63891602,   0.        ,   0.        , ...,  16.13916016,
         20.58654785,  12.05041504],
       [ -8.29736328,   0.        ,  42.11193848, ...,   0.94616699,
          3.57092285,   6.35217285],
       [  1.85791016,   0.        ,  11.20837402, ...,  17.1328125 ,
         13.16650391,  -2.81518555],
       ..., 
       [-13.7175293 ,   0.        ,  47.6862793 , ...,   0.        ,
        -21.03918457,  15.2722168 ],
       [ 17.33740234,  27.43493652, -20.22363281, ...,   0.        ,
         -2.67407227,  12.81750488],
       [  6.32019043,  18.85083008, -27.16552734, ...,  27.34533691,
         -8.68847656,   0.        ]], dtype=float32)

特定のピクセルの値を書き出してみます。

In [4]:
print (data[0, 5])

32.5959


これは(x, y) = (6, 1)のピクセルであることに注意してください。
- Pythonではインデックスは0から始まる
- numpyのndarrayではFITSの二次元データは(y, x)の順

idf0011.fitsをds9で直接オープンして、(6, 1)のピクセル値を読み取って確認してみてください。(ds9のメニューバーのanalysis>pixel table を使うと便利です。)

#### numpyの関数で処理

FITSの二次元データはndarrayとして読み出されるので、numpyの関数が使えます。

In [5]:
import numpy as np

In [6]:
np.median(data), np.std(data), np.min(data), np.max(data)

(2.8615723, 199.40004, -82570.391, 47768.0)

#### _補足_
上限値と下限値を適切に設定し、3-sigmaクリッップすると標準偏差がもう少し小さくなります。


In [7]:
xx = np.where(np.fabs(data) < 1000)  #  値が+/-1000以内のところだけ抽出
med = np.median(data[xx])
std = np.std(data[xx])

xx = np.where(np.fabs(data - med)  < 3 * std)  #  3-sigma clip
med = np.median(data[xx])
std = np.std(data[xx])
print ('{:.2f} {:.2f}'.format(med, std))   #   format()関数使ってみた


2.79 20.26


#### データの加工  

IRAFは長期間にわたって多くの人たちに利用されてきたので、プログラムのバグがかなり修正されてきています。
こういうのを、いい意味で「枯れたシステム」ということがあります。
枯れたシステムのIRAFの関数はとてもありがたい存在なのですが、画像の足し算、引き算、割り算くらいなら、
IRAFを介さずにpythonのモジュールだけで済ませるほうが便利なことが多いです。  

astropy.io.fitsでFITSをndarrayとして読み取り、講習2で行った1次処理のダーク引き、フラット割り、スカイバイアス引きの部分を、
ndarrayの演算として行ってみます。 

In [8]:
rdata = fits.getdata('sample_data2/ir0011.fits')
dark = fits.getdata('dark5.fits')
flat = fits.getdata('flat.fits')
skybias = fits.getdata('skybias.fits')

0での割り算に注意します。IRAFでは何か裏で工夫してたのですが、ここでは明示して処理しておきます。  
ピクセル値が0のところを-9999におきかえます。

In [9]:
xx = np.where( np.abs(flat) == 0.0 ) 
flat[xx] = -9999.

IRAFの場合と違い、一行で処理の式が書けます。また、途中の暫定的なファイルを介在せずに処理ができます。

In [10]:
ndata= ( rdata - dark ) / flat - skybias

結果をファイルに書き出すときには、**fits.writeto()**関数を使います。  
**fits.writeto('ファイル名.fits', ndarrayデータ)** 

In [11]:
fits.writeto('new0011.fits', ndata)

はたして、idf0011.fitsと同じでしょうか？  ds9で見比べてみてください。

あるいは、ちゃんと比較するには、(idf0011.fitsのデータは上でdataとして読み込んでいるので）、dataとndataの差分の統計をみてみましょう。

In [12]:
diff = data - ndata

In [14]:
np.median(diff), np.std(diff)

(0.0, 2.7136393e-06)

IRAFで処理したものも、非IRAFで処理したものも同じ結果が得られたようですね。